In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
train = pd.read_csv('../input/paddy-disease-classification/train.csv')
sub = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')

In [ ]:
diseases_types = sorted(list(set(train['label'])))
diseases_types

In [ ]:
train_data= tf.keras.preprocessing.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=diseases_types,
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=True,
                                                        validation_split=0.2,
                                                        subset='training',
                                                        seed=42)

val_data= tf.keras.preprocessing.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=diseases_types,   
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=True,
                                                        validation_split=0.2,
                                                        subset='validation',
                                                        seed=42)

In [ ]:
train_data

In [ ]:

# predict with resnet architecture
resnet_model=tf.keras.models.Sequential()
pretrained_model=tf.keras.applications.ResNet50(include_top=False,input_shape=(680,480,3),pooling='avg',classes=10,weights='imagenet')
    
for layer in pretrained_model.layers:
    layer.trainable=False
      
resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.Flatten())
resnet_model.add(tf.keras.layers.Dense(512,activation='relu'))
resnet_model.add(tf.keras.layers.Dense(99,activation='relu'))
resnet_model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [ ]:
# predict with resnet architecture
efficientnet_model=tf.keras.models.Sequential()
pretrained_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False,input_shape=(680,480,3),pooling='avg',classes=10,weights='imagenet')
    
for layer in pretrained_model.layers:
    layer.trainable=False
      
efficientnet_model.add(pretrained_model)
efficientnet_model.add(tf.keras.layers.Flatten())
efficientnet_model.add(tf.keras.layers.Dense(512,activation='relu'))
efficientnet_model.add(tf.keras.layers.Dense(99,activation='relu'))
efficientnet_model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [ ]:
tf.keras.utils.plot_model(efficientnet_model)

In [ ]:
val_data

In [ ]:
resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
resnet_model.fit(train_data,validation_data=val_data,epochs=25,)

In [ ]:
efficientnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
efficientnet_model.fit(train_data,validation_data=val_data,epochs=25)

In [ ]:
test_data=tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/test_images/",
                                                        labels=None,
                                                        label_mode=None,
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=False)

In [ ]:
predict_res=tf.argmax(resnet_model.predict(test_data),axis=-1)
predict_res

In [ ]:
predictions=[]
for i in predict_res:
    predictions.append(diseases_types[i])

In [ ]:
sub=sub.drop(columns='label',axis=1)
sub

In [ ]:
sub

In [ ]:
submission=pd.DataFrame({'image_id':sub['image_id'],'label':predictions})

In [ ]:
submission.to_csv('paddy_docto2.csv',index=None)